In [1]:
#@title Huggingface Login
#@markdown huggingface weight 를 이용하고 싶다면 로그인 필수
from google.colab import userdata
import os

os.environ['HF_WRITE_TOKEN'] = userdata.get('HF_WRITE_TOKEN')

!huggingface-cli login --add-to-git-credential --token $HF_WRITE_TOKEN


Token is valid (permission: write).
Cannot authenticate through git-credential as no helper is defined on your machine.
You might have to re-authenticate when pushing to the Hugging Face Hub.
Run the following command in your terminal in case you want to set the 'store' credential helper as default.

git config --global credential.helper store

Read https://git-scm.com/book/en/v2/Git-Tools-Credential-Storage for more details.
Token has not been saved to git credential helper.
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [2]:
#@markdown install with openvino
%%sh
# apt-get update  -y
# apt-get install -y gcc-12 g++-12
# update-alternatives --install /usr/bin/gcc gcc /usr/bin/gcc-12 10 --slave /usr/bin/g++ g++ /usr/bin/g++-12
# pip install --upgrade pip
# pip install wheel packaging ninja "setuptools>=49.4.0" numpy
git clone https://github.com/vllm-project/vllm.git
cd vllm && pip install -r requirements-build.txt --extra-index-url https://download.pytorch.org/whl/cpu
pip install gguf
PIP_EXTRA_INDEX_URL="https://download.pytorch.org/whl/cpu https://storage.openvinotoolkit.org/simple/wheels/pre-release" VLLM_TARGET_DEVICE=openvino python -m pip install -v .

Output hidden; open in https://colab.research.google.com to view.

In [3]:
#@markdown install with cpu
%%sh
# apt-get update  -y
apt-get install -y gcc-12 g++-12
update-alternatives --install /usr/bin/gcc gcc /usr/bin/gcc-12 10 --slave /usr/bin/g++ g++ /usr/bin/g++-12
# pip install --upgrade pip
pip install wheel packaging ninja "setuptools>=49.4.0" numpy
git clone https://github.com/vllm-project/vllm.git
cd vllm && pip install -v -r requirements-cpu.txt --extra-index-url https://download.pytorch.org/whl/cpu
VLLM_TARGET_DEVICE=cpu python setup.py install

Reading package lists...
Building dependency tree...
Reading state information...
The following additional packages will be installed:
  cpp-12 libasan8 libgcc-12-dev libstdc++-12-dev libtsan2
Suggested packages:
  gcc-12-locales cpp-12-doc g++-12-multilib gcc-12-doc gcc-12-multilib libstdc++-12-doc
The following NEW packages will be installed:
  cpp-12 g++-12 gcc-12 libasan8 libgcc-12-dev libstdc++-12-dev libtsan2
0 upgraded, 7 newly installed, 0 to remove and 45 not upgraded.
Need to get 54.5 MB of archives.
After this operation, 196 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 cpp-12 amd64 12.3.0-1ubuntu1~22.04 [10.8 MB]
Get:2 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 libasan8 amd64 12.3.0-1ubuntu1~22.04 [2,442 kB]
Get:3 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 libtsan2 amd64 12.3.0-1ubuntu1~22.04 [2,477 kB]
Get:4 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 libgcc-12-dev amd64

fatal: destination path 'vllm' already exists and is not an empty directory.
/usr/local/lib/python3.10/dist-packages/setuptools/_distutils/cmd.py:66: SetuptoolsDeprecationWarning: setup.py install is deprecated.
!!

        ********************************************************************************
        Please avoid running ``setup.py`` directly.
        Instead, use pypa/build, pypa/installer or other
        standards-based tools.

        See https://blog.ganssle.io/articles/2021/10/setup-py-deprecated.html for details.
        ********************************************************************************

!!
  self.initialize_options()
/usr/local/lib/python3.10/dist-packages/setuptools/_distutils/cmd.py:66: EasyInstallDeprecationWarning: easy_install command is deprecated.
!!

        ********************************************************************************
        Please avoid running ``setup.py`` and ``easy_install``.
        Instead, use pypa/build, pypa/install

In [4]:
#@markdown colab installation
!pip install pynvml
!VLLM_TARGET_DEVICE=cpu pip install vllm

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.1/53.1 kB 1.8 MB/s eta 0:00:00


In [5]:
from vllm import LLM, SamplingParams
import os

os.environ["VLLM_CPU_KVCACHE_SPACE"] = "5"
os.environ["VLLM_CPU_OMP_THREADS_BIND"] = "0-29"

model  = LLM(
    # model = "KISTI-KONI/KONI-Llama3-8B-Instruct-20240729",
    model = "Gunulhona/Llama-Merge-Small",
    max_model_len=1024
    )

WARNING 08-25 23:36:47 _custom_ops.py:17] Failed to import from vllm._C with ModuleNotFoundError("No module named 'vllm._C'")


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/939 [00:00<?, ?B/s]

INFO 08-25 23:36:55 llm_engine.py:194] Initializing an LLM engine (v0.5.5) with config: model='Gunulhona/Llama-Merge-Small', speculative_config=None, tokenizer='Gunulhona/Llama-Merge-Small', skip_tokenizer_init=False, tokenizer_mode=auto, revision=None, rope_scaling=None, rope_theta=None, tokenizer_revision=None, trust_remote_code=False, dtype=torch.bfloat16, max_seq_len=1024, download_dir=None, load_format=LoadFormat.AUTO, tensor_parallel_size=1, pipeline_parallel_size=1, disable_custom_all_reduce=False, quantization=None, enforce_eager=False, kv_cache_dtype=auto, quantization_param_path=None, device_config=cpu, decoding_config=DecodingConfig(guided_decoding_backend='outlines'), observability_config=ObservabilityConfig(otlp_traces_endpoint=None, collect_model_forward_time=False, collect_model_execute_time=False), seed=0, served_model_name=Gunulhona/Llama-Merge-Small, use_v2_block_manager=False, num_scheduler_steps=1, enable_prefix_caching=False)


tokenizer_config.json:   0%|          | 0.00/55.4k [00:00<?, ?B/s]

...

model-00016-of-00017.safetensors:   0%|          | 0.00/990M [00:00<?, ?B/s]

model-00017-of-00017.safetensors:   0%|          | 0.00/201M [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/17 [00:00<?, ?it/s]

Using framework PyTorch: 2.4.0+cpu
Overriding 1 configuration item(s)
	- use_cache -> True
We detected that you are passing `past_key_values` as a tuple and this is deprecated and will be removed in v4.43. Please use an appropriate `Cache` class (https://huggingface.co/docs/transformers/v4.41.3/en/internal/generation_utils#transformers.Cache)
/usr/local/lib/python3.10/dist-packages/optimum/exporters/openvino/model_patcher.py:479: TracerWarning: Converting a tensor to a Python boolean might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will be treated as a constant in the future. This means that the trace might not generalize to other inputs!
  if sequence_length != 1:


INFO 08-25 23:52:33 openvino_executor.py:73] # CPU blocks: 1024


In [ ]:
import gc
import os
from vllm import LLM, SamplingParams

os.environ["VLLM_USE_MODELSCOPE"] = "True"

prompt = """
---
[대화]
의사: 안녕하세요, 선생님.
환자: 안녕하세요, 의사 선생님.
의사: 마흔 넷이시죠?
환자: 네, 선생님.
의사: 좋아요, 오늘은 무슨 문제가 있는 것 같나요?
환자: 의사 선생님, 한동안 허리 통증이 있었습니다.
의사: 통증이 다리로 내려가나요?
환자: 네, 오른쪽 허벅지에도 통증이 있습니다.
의사: 이 통증과 관련된 부상이 있습니까?
환자: 네, 1994년에 사고가 있었습니다.
의사: 최초 부상 당시의 서류나 의료 기록이 있습니까?
환자: 아니요, 오늘은 없습니다.
의사: 직업이 어떻게 되십니까?
환자: 지금은 타코벨에서 일합니다. 산재 보험 청구가 열려 있습니다.
의사: 거기서 일하다가 통증이 재발했죠?
환자: 네, 맞습니다.
의사: 마지막으로 이곳에서 진료를 받은 것이 언제였는지 기억하십니까?
환자: 음, 네, 4월 12일 2005년이었습니다.
의사: 10이 상상할 수 있는 최악의 통증이라면, 마지막 방문 시 통증은 10점 만점에 어느 정도였습니까?
환자: 음, 10점 만점에 8점 정도였어요.
의사: 이 통증 때문에 약을 복용하셨나요?
환자: 음, 지난번 방문했을 때 메드롤 도스팩을 처방받았습니다.
의사: 도세팍에 통증이 어떻게 반응했나요?
환자: 통증이 10점 만점에 4~5점 정도로 줄었습니다.
의사: 통증이 있는 곳을 가리켜 주시겠습니까?
환자: 네, 바로 여기입니다.
의사: 여기 이 밴드요?
환자: 네, 바로 그 자리입니다.
의사: 좋아요, 여기는 요추 4번과 천골 사이입니다. 오른쪽 다리 통증을 어떻게 설명하시겠습니까?
환자: 지금은 간헐적이고 미미하며 항상 있는 것은 아닙니다.
의사: 허리 수술을 받은 적이 있습니까?
환자: 음, 네, 1990년에 한 번, 1994년에 한 번 두 번 척추 절제술을 받았습니다. 잠깐만요, 그 사이에 디스크 절제술도 받았어요.
의사: 어디에 초점이 맞춰졌는지 아십니까?
환자: L 4 L 5번이었습니다.
의사: 허리에 대한 영상 촬영은 하셨나요?
환자: 네, 10월 18일 2004년에 MRI를 찍었습니다. 여기 보고서가 있습니다.
의사: 좋아요, 이것은 다단계 퇴행성 변화를 보여 주며, L 2 L 3, L 3 L 4, L 5 S1에서 신경 침범이 없는 다단계 퇴행성 변화를 보여 주며, 이는 양호합니다.
환자: 그게 무슨 뜻인가요, 의사 선생님?
의사: 요약하자면, 허리에 상당한 양의 관절염이 있다는 뜻입니다.
환자: 네, M R 골수 조영술도 받았는데 여기 보고서가 있습니다.
의사: 좋아요, 요추 3번에서 심한 척추관 협착증이 보이지만 인공물일 수도 있습니다.
환자: 그게 무슨 뜻인가요?
의사: 이 소견은 잘못된 해석일 수 있습니다.
 ---
[요약]
 *
"""

output_str = model.generate(
    prompts=prompt,
    sampling_params=SamplingParams(
        temperature=0.5,
        top_p=0.9,
        max_tokens=200,
    ))[0].outputs[0].text

print(f'''
{output_str}
''')

gc.collect()

Processed prompts:   0%|          | 0/1 [00:00<?, ?it/s, est. speed input: 0.00 toks/s, output: 0.00 toks/s]

 Blue whale. The blue whale is not only the largest animal but also the largest
